In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [907 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
G

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-23 03:11:46--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.23MB/s    in 0.8s    

2022-04-23 03:11:48 (1.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [ ]:
# Filter the vine_table DataFrame to show total_votes > 20
filtered_vine_df = vine_df.filter('total_votes >= 20')
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2MDI1TP46VSYD|          3|            9|         21|   N|                Y|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|     

In [ ]:
# Filter DataFrame to show >= 50% of helpful_votes/total_votes
votes50_df = filtered_vine_df.withColumn('GreaterThan50', filtered_vine_df['helpful_votes']/filtered_vine_df['total_votes']).filter('GreaterThan50 >= 0.5')
votes50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     GreaterThan50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|0.8518518518518519|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|               1.0|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|              0.95|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|0.7283950617283951|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|0.9565217391304348|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|               1.0|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|               1.0|
|R29IYHPYD14AGI|          3|  

In [ ]:
# Filter DataFrame to show Vine users
vine_votes_df = votes50_df.filter(votes50_df['vine'] == "Y")
vine_votes_df.show()
print(vine_votes_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     GreaterThan50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|              0.95|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|0.9565217391304348|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|               1.0|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|0.9259259259259259|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|             0.875|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|0.8928571428571429|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|0.9090909090909091|
|R2H4VEFWHP50M3|          5|  

In [ ]:
# Filter DataFrame to show Non-Vine users
regular_votes_df = votes50_df.filter(votes50_df['vine'] == "N")
regular_votes_df.show()
print(regular_votes_df.count())

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     GreaterThan50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|0.8518518518518519|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|               1.0|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|0.7283950617283951|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|               1.0|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|               1.0|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|               1.0|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|               1.0|
|R2GHOWIWDVLN18|          1|  

In [ ]:
print("Total (Helpful) Vine Reviews:  %f" % vine_votes_df.count())
print("Total (Helpful) Non-Vine Reviews: %f" % regular_votes_df.count())

Total (Helpful) Vine Reviews:  1266.000000
Total (Helpful) Non-Vine Reviews: 62028.000000


In [ ]:
five_star_vine_df = vine_votes_df.filter(vine_votes_df["star_rating"] == '5')
print("Total (Helpful) 5-Star Vine Reviews: %f" % five_star_vine_df.count())

five_star_regular_df = regular_votes_df.filter(regular_votes_df["star_rating"] == '5')
print("Total (Helpful) 5-Star Non-Vine Reviews: %f" % five_star_regular_df.count())

Total (Helpful) 5-Star Vine Reviews: 432.000000
Total (Helpful) 5-Star Non-Vine Reviews: 29982.000000


In [ ]:
vine_percent_df = (five_star_vine_df.count()/vine_votes_df.count()) * 100
print("Percent of Vine 5-Star Votes: %f" % vine_percent_df)

regular_percent_df = (five_star_regular_df.count()/regular_votes_df.count()) * 100
print("Percent of Non-Vine 5-Star Votes: %f" % regular_percent_df)

Percent of Vine 5-Star Votes: 34.123223
Percent of Non-Vine 5-Star Votes: 48.336235
